In [1]:
import requests
import json

In [2]:
# Download the data
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)  # Load data


In [3]:
print(type(data), len(data))

<class 'list'> 10


In [6]:
print(json.dumps(data[0], indent=4))

{
    "Category": "SCIENCE",
    "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
    "Answer": "Liver"
}


In [9]:
def json_print(data, indent=4):
    print(json.dumps(data, indent=indent))

In [10]:
json_print(data)

[
    {
        "Category": "SCIENCE",
        "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
        "Answer": "Liver"
    },
    {
        "Category": "ANIMALS",
        "Question": "It's the only living mammal in the order Proboseidea",
        "Answer": "Elephant"
    },
    {
        "Category": "ANIMALS",
        "Question": "The gavial looks very much like a crocodile except for this bodily feature",
        "Answer": "the nose or snout"
    },
    {
        "Category": "ANIMALS",
        "Question": "Weighing around a ton, the eland is the largest species of this animal in Africa",
        "Answer": "Antelope"
    },
    {
        "Category": "ANIMALS",
        "Question": "Heaviest of all poisonous snakes is this North American rattlesnake",
        "Answer": "the diamondback rattler"
    },
    {
        "Category": "SCIENCE",
        "Question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new on

In [12]:
!pip install -U weaviate-client

   ---------------------------------------- 0.0/368.3 kB ? eta -:--:--
   ----- ---------------------------------- 51.2/368.3 kB 1.3 MB/s eta 0:00:01
   -------------------------------------- - 358.4/368.3 kB 4.4 MB/s eta 0:00:01
   ---------------------------------------- 368.3/368.3 kB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/43.3 kB ? eta -:--:--
   ---------------------------------------- 43.3/43.3 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/225.1 kB ? eta -:--:--
   --------------------------------------- 225.1/225.1 kB 14.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   --------- ------------------------------ 1.0/4.3 MB 33.4 MB/s eta 0:00:01
   ------------------- -------------------- 2.1/4.3 MB 26.8 MB/s eta 0:00:01
   ---------------------------- ----------- 3.1/4.3 MB 24.5 MB/s eta 0:00:01
   -------------------------------------- - 4.1/4.3 MB 23.9 MB/s eta 0:00:01
   -----------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 5.27.4 which is incompatible.
opentelemetry-proto 1.26.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.27.4 which is incompatible.
tensorboard 2.17.0 requires protobuf!=4.24.0,<5.0.0,>=3.19.6, but you have protobuf 5.27.4 which is incompatible.
tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.27.4 which is incompatible.


In [15]:
import weaviate
from weaviate.embedded import EmbeddedOptions
import os

In [67]:
#Start up an instance of Weaviate
client = weaviate.connect_to_weaviate_cloud(cluster_url='https://kadg1vjettw3hkvszkev3g.c0.us-east1.gcp.weaviate.cloud',  # Replace with your Weaviate Cloud URL
                         auth_credentials=weaviate.auth.AuthApiKey(
                                api_key = 'fFw62ouvsYZZMm3EfnAG9KXrQyno8lMu067i',),
                        #additional_headers={
                        # "X-OpenAI-Api-Key": "<OPENAI_API_KEY>" 
                        #}
                        )

In [70]:
# Check connection
client.is_ready()

True

In [89]:
response = client.collections.list_all(simple=True)
if response['Question']:
    client.collections.delete("Question") 

In [92]:
import weaviate.classes as wvc

# Create the collection. Weaviate's autoschema feature will infer properties when importing.


questions = client.collections.create(
        "Question",
        vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),
    )

In [93]:
with client.batch.dynamic() as batch:
    question_objs = list()
    for i, d in enumerate(data):
        question_objs.append(wvc.data.DataObject(
            properties={
                "answer": d["Answer"],
                "question": d["Question"],
                "category": d["Category"],
            }
        ))

    questions = client.collections.get("Question")
    questions.data.insert_many(question_objs)    # This uses batching under the hood

WeaviateInsertManyAllFailedError: Every object failed during insertion. Here is the set of all errors: API Key: no api key found neither in request header: X-Openai-Api-Key nor in environment variable under OPENAI_APIKEY

In [94]:

#Check how many objects we've loaded into the database

questions = client.collections.get("Question")
response = questions.query.near_text(
    query="animal",
    limit=2
)

for o in response.objects:
    print(o.properties)  # Object properties

WeaviateQueryError: Query call with protocol GRPC search failed with message <AioRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "explorer: get class: vectorize params: vectorize params: vectorize params: vectorize keywords: remote client vectorize: API Key: no api key found neither in request header: X-Openai-Api-Key nor in environment variable under OPENAI_APIKEY"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2024-08-28T03:52:54.272634+00:00", grpc_status:2, grpc_message:"explorer: get class: vectorize params: vectorize params: vectorize params: vectorize keywords: remote client vectorize: API Key: no api key found neither in request header: X-Openai-Api-Key nor in environment variable under OPENAI_APIKEY"}"
>.

In [95]:

#Extract and show any 3 questions and answers

json_print(client.collections.get("Question", ["question","answer"]).with_limit(3).do())

AttributeError: 'WeaviateClient' object has no attribute 'query'

In [96]:
json_print(data)

[
    {
        "Category": "SCIENCE",
        "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
        "Answer": "Liver"
    },
    {
        "Category": "ANIMALS",
        "Question": "It's the only living mammal in the order Proboseidea",
        "Answer": "Elephant"
    },
    {
        "Category": "ANIMALS",
        "Question": "The gavial looks very much like a crocodile except for this bodily feature",
        "Answer": "the nose or snout"
    },
    {
        "Category": "ANIMALS",
        "Question": "Weighing around a ton, the eland is the largest species of this animal in Africa",
        "Answer": "Antelope"
    },
    {
        "Category": "ANIMALS",
        "Question": "Heaviest of all poisonous snakes is this North American rattlesnake",
        "Answer": "the diamondback rattler"
    },
    {
        "Category": "SCIENCE",
        "Question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new on